In [2]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import f_regression
from sklearn.cross_validation import cross_val_score, cross_val_predict

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
diabetes = datasets.load_diabetes()

In [4]:
# View dict elements
diabetes.keys()

# View Data
print(diabetes.data.shape)
col_names = "age sex bmi map tc ldl hdl tch ltg glu".split()
data_frame = pd.DataFrame(diabetes.data, columns=col_names)
y = diabetes.target
sns.pairplot(data_frame)

(442, 10)


In [5]:
# Check for extreme/suspicious values
for col_name in col_names:
    print(col_name,":",min(data_frame[col_name]),np.percentile(data_frame[col_name], 
        [25, 50, 75]),max(data_frame[col_name]))

age : -0.107225631607358 [-0.03729927  0.00538306  0.03807591] 0.110726675453815
sex : -0.044641636506989 [-0.04464164 -0.04464164  0.05068012] 0.0506801187398187
bmi : -0.0902752958985185 [-0.03422907 -0.00728377  0.03124802] 0.17055522598066
map : -0.112399602060758 [-0.03665645 -0.00567061  0.03564384] 0.132044217194516
tc : -0.126780669916514 [-0.03424784 -0.00432087  0.02835801] 0.153913713156516
ldl : -0.115613065979398 [-0.0303584  -0.00381907  0.02984439] 0.198787989657293
hdl : -0.10230705051742 [-0.03511716 -0.00658447  0.0293115 ] 0.181179060397284
tch : -0.076394503750001 [-0.03949338 -0.00259226  0.03430886] 0.185234443260194
ltg : -0.126097385560409 [-0.03324879 -0.00194763  0.03243323] 0.133598980013008
glu : -0.137767225690012 [-0.03317903 -0.0010777   0.02791705] 0.135611830689079


In [6]:
regr = linear_model.LinearRegression()

# LOO Cross-Validation 
scores = cross_val_score(regr, data_frame, y, cv=5)
print(scores)

[0.42955643 0.52259828 0.4826784  0.42650827 0.55024923]


First try LOO Cross-Validation using all possible models with 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 variables. For each run, calculate $$RMSE = \sqrt{\frac{1}{n}\sum \limits_{i=1}^n (y_i - \widehat{y}_{(-i)})^2}$$

Then try a forward selection algorithm and see what it picks. Use the AIC criterion (or BIC).

If time try LASSO for variable selection

In [11]:
import itertools
from sklearn.model_selection import LeaveOneOut


def looRMSE(X, y):
    loo = LeaveOneOut()
    regr = linear_model.LinearRegression()
    rmse = 0
    for train, test in loo.split(X):
        regr.fit(X[train], y[train])
        rmse += (regr.predict(X[test]) - y[test])**2
        print(rmse)
    
    rmse = np.sqrt(1./X.shape[1]*rmse)
    return rmse    
        
     
tested = []
result = []
for i in range(len(col_names)):
    for j in range(1, i + 1):
        testCombos = itertools.combinations(np.arange(i), j)
        for c in testCombos:
            if set(c) in tested:
                continue
            tested.append(set(c))
            testSet = data_frame.values[:, c]
            result.append(looRMSE(testSet, y))
            print("{}, {}".format(set(c), result[-1]))
            break
        

[162.89184996]
[6051.13469051]
[7452.66780404]
[14142.56525706]
[14496.53301499]
[15238.15278973]
[15238.2445002]
[27149.18689674]
[30178.67241944]
[62849.49342452]
[63337.33121872]
[71741.78146786]
[72224.45087486]
[73204.19116165]
[75521.00528701]
[76751.98687069]
[76994.7130514]
[77887.37023345]
[79794.78894303]
[80382.68804677]
[85215.92633009]
[91283.04465531]
[94729.70917829]
[101036.84858043]
[103694.10944474]
[108707.99860696]
[109022.23283227]
[112634.3551691]
[114027.36795468]
[126392.27237831]
[126416.38527237]
[133842.4369294]
[165885.86525271]
[171472.78215712]
[179995.60338736]
[184264.25473931]
[196207.22508476]
[212322.69942477]
[222456.56517206]
[226263.55632957]
[229168.16427618]
[233731.84665472]
[239106.42369495]
[243511.03104718]
[252249.2083386]
[263853.65795133]
[266913.32479985]
[267102.61485552]
[276749.80412865]
[276756.58477305]
[276814.90719536]
[279837.28698749]
[285840.81086043]
[287907.21718173]
[289933.46061756]
[290064.61903739]
[297766.2015019]
[309247

In [8]:
# View histogram of outcome variable
plt.hist(np.sqrt(y_train))

NameError: name 'y_train' is not defined

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(diabetes_X_train, diabetes_y_train)

In [ ]:
data_frame.shape[0]

In [ ]:
y